In [65]:
# preparation of the environment
%load_ext autoreload
%autoreload 2

import os
from os import path
# set the repository to the git repository
cwd = os.getcwd().split(os.path.sep)
while cwd[-1] != "stage_4_gm":
    os.chdir("..")
    cwd = os.getcwd().split(os.path.sep)
print(">> the git rep : ", end="")
print(os.getcwd())

# the folder where we will save our data
foler_name = "reg_mul=0"
plots_folder = os.path.join(os.getcwd(), '.cache', 'plots')
graph_folder = path.join(plots_folder, foler_name)
if not path.exists(graph_folder):
    os.mkdir(graph_folder)

print(f">> the plots location : {graph_folder}")

import pandas as pd
import torch
import numpy as np
import pickle
from tqdm import tqdm
from tabulate import tabulate
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import entropy

from regularize_training_bert import BertNliRegu
from custom_data_set import SnliDataset
from custom_data_set import test_dir, dev_dir, train_dir
from torch.utils.data import DataLoader

# --> from this environment
from attention_algorithms.attention_metrics import attention_score
from attention_algorithms.raw_attention import RawAttention
from attention_algorithms.attention_metrics import normalize_attention

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
>> the git rep : C:\Users\loicf\Documents\IRISA\stage_4_gm\stage_4_gm
>> the plots location : C:\Users\loicf\Documents\IRISA\stage_4_gm\stage_4_gm\.cache\plots\reg_mul=0


## The metrics and the Rows

In [66]:
from sklearn.metrics import f1_score, precision_score, recall_score, roc_auc_score, average_precision_score, auc
def jaccard_score(y, y_hat):
    num = np.dot(y,y_hat)
    den = np.sum(y) + np.sum(y_hat) - np.dot(y, y_hat)
    return num/den

In [67]:
def create_row(y, y_hat, metrics):
    row = []
    tr = np.linspace(0, 1, 50)
    for m in metrics :
        if m != "roc_auc_score" and m != "jaccard_score" and m != "average_precision_score":
            ar = [] # --> calculus of the metrics
            for t in tr:
                buff = 1 * (np.array(y_hat)>=t)
                ar.append(eval(m)(y, buff))
            row.append(auc(tr, ar))
        else:
            row.append(eval(m)(y, y_hat))

    return row

## The baseline map

In [68]:
dir = os.path.join(graph_folder, "cls_map_baseline.pickle")
with open(dir, "rb") as f:
    d = pickle.load(f)
    y = d["y"]
    y_hat = d["y_hat"]
rows = []
evaluation_metrics = ["f1_score", "precision_score", "recall_score", "roc_auc_score", "jaccard_score",
                      "average_precision_score"]

# set the header for the table of statistics
h1 = ["AU_" + x + "_curve" for x in ["f1", "PR", "RC"]]

h2 = ["roc_auc_score", "jaccard_score", "AUPRC"]
rows.append(h1 + h2)

rows.append(create_row(y, y_hat, evaluation_metrics))
print(tabulate(rows, headers="firstrow"))

  AU_f1_curve    AU_PR_curve    AU_RC_curve    roc_auc_score    jaccard_score     AUPRC
-------------  -------------  -------------  ---------------  ---------------  --------
     0.359462       0.516891       0.343315         0.746577         0.233119  0.432513


In [69]:
from openpyxl import load_workbook, Workbook
from openpyxl.worksheet.table import Table, TableStyleInfo

xls_dir = os.path.join(plots_folder, "dash_board.xlsx")
wb = None
if os.path.exists(xls_dir):
    wb = load_workbook(xls_dir)
else :
    wb = Workbook()

ws = None
if "sep_vs_cls_no_reg" not in wb.sheetnames:
    ws = wb.create_sheet("sep_vs_cls_no_reg")

ws = wb["sep_vs_cls_no_reg"] # get the work sheet

ws.append(["CLS map"])
for r in rows :
    ws.append(r)

wb.save(xls_dir)
wb.close()

## The SEP map

In [70]:
dir = os.path.join(graph_folder, "sep_map.pickle")
with open(dir, "rb") as f:
    d = pickle.load(f)
    y = d["y"]
    y_hat = d["y_hat"]
rows = []
evaluation_metrics = ["f1_score", "precision_score", "recall_score", "roc_auc_score", "jaccard_score",
                      "average_precision_score"]

# set the header for the table of statistics
h1 = ["AU_" + x + "_curve" for x in ["f1", "PR", "RC"]]

h2 = ["roc_auc_score", "jaccard_score", "AUPRC"]
rows.append(h1 + h2)

rows.append(create_row(y, y_hat, evaluation_metrics))
print(tabulate(rows, headers="firstrow"))

  AU_f1_curve    AU_PR_curve    AU_RC_curve    roc_auc_score    jaccard_score     AUPRC
-------------  -------------  -------------  ---------------  ---------------  --------
     0.257856       0.407304       0.236514           0.7131         0.165973  0.346711


In [71]:
from openpyxl import load_workbook, Workbook
from openpyxl.worksheet.table import Table, TableStyleInfo

xls_dir = os.path.join(plots_folder, "dash_board.xlsx")
wb = None
if os.path.exists(xls_dir):
    wb = load_workbook(xls_dir)
else :
    wb = Workbook()

ws = None
if "sep_vs_cls_no_reg" not in wb.sheetnames:
    ws = wb.create_sheet("sep_vs_cls_no_reg")

ws = wb["sep_vs_cls_no_reg"] # get the work sheet

ws.append(["SEP map"])
for r in rows :
    ws.append(r)

wb.save(xls_dir)
wb.close()

## Comb map

In [72]:
dir = os.path.join(graph_folder,"comb_map.pickle")
with open(dir, "rb") as f:
    d = pickle.load(f)
    y = d["y"]
    y_hat = d["y_hat"]

In [73]:
rows = []
evaluation_metrics = ["f1_score", "precision_score", "recall_score", "roc_auc_score", "jaccard_score", "average_precision_score"]

# set the header for the table of statistics
h1 = ["au_"+x+"_curve" for x in ["f1", "PR", "RC"]]

h2 = ["roc_auc_score", "jaccard_score", "AUPRC"]
rows.append(h1 + h2)

rows.append(create_row(y, y_hat, evaluation_metrics))
print(tabulate(rows, headers="firstrow"))

  au_f1_curve    au_PR_curve    au_RC_curve    roc_auc_score    jaccard_score    AUPRC
-------------  -------------  -------------  ---------------  ---------------  -------
     0.303823       0.456789        0.28628         0.736825          0.19817  0.38953


In [74]:
from openpyxl import load_workbook, Workbook
from openpyxl.worksheet.table import Table, TableStyleInfo

xls_dir = os.path.join(plots_folder, "dash_board.xlsx")
wb = None
if os.path.exists(xls_dir):
    wb = load_workbook(xls_dir)
else :
    wb = Workbook()

ws = None
if "sep_vs_cls_no_reg" not in wb.sheetnames:
    ws = wb.create_sheet("sep_vs_cls_no_reg")

ws = wb["sep_vs_cls_no_reg"] # get the work sheet

ws.append(["CLS + SEP"])
for r in rows :
    ws.append(r)

wb.save(xls_dir)
wb.close()